In [2]:
from tensorflow import keras
from tensorflow.keras.layers import Conv1D,MaxPool1D
import numpy as np

### 函数generate_time_series()生成的时间序列

In [5]:
'''生成一批时间序列数据，每个时间序列由两个正弦波的和以及一些噪声构成'''
def generate_time_series(batch_size,n_steps):
    freq1,freq2,offsets1,offsets2=np.random.rand(4,batch_size,1) #每个时间步只有一个值
    
    time=np.linspace(0, 1, n_steps) #生成时间步数组
    series=0.5*np.sin((time-offsets1)*(freq1*10+10)) #wave 1
    series+=0.2*np.sin((time-offsets2)*(freq2*20+20)) # +wave 2
    series+=0.1*(np.random.rand(batch_size,n_steps)-0.5) # +noise
    return series[...,np.newaxis].astype(np.float32) #返回 [批次大小，时间步数，1]，每个序列是2个正弦波之和（固定强度+随机频率和相位）+噪声
#在原始数组 series 的末尾添加了一个新的轴，从而改变了数组的维度
#series 是一个二维数组，形状为 (batch_size, n_steps)

#其形状是 批次大小 时间步数 维度 

### 创建训练集、验证集和测试集

In [6]:
n_steps = 50
series = generate_time_series(10000, n_steps + 10)
X_train = series[:7000, :n_steps]
X_valid = series[7000:9000, :n_steps]
X_test = series[9000:, :n_steps]
Y = np.empty((10000, n_steps, 10))
for step_ahead in range(1, 10 + 1):
    Y[..., step_ahead - 1] = series[..., step_ahead:step_ahead + n_steps, 0]
Y_train = Y[:7000]
Y_valid = Y[7000:9000]
Y_test = Y[9000:]

### 使用1D卷积层处理序列

In [7]:
def last_time_step_mse(Y_true, Y_pred):
    return keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])

In [8]:
model = keras.models.Sequential([
    keras.layers.Conv1D(filters=20, kernel_size=4, strides=2, padding="valid",
                        input_shape=[None, 1]),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.GRU(20, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(10))
])

model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train[:, 3::2], epochs=20,
                    validation_data=(X_valid, Y_valid[:, 3::2]))

Epoch 1/20
219/219 [==============================] - 4s 10ms/step - loss: 0.0667 - last_time_step_mse: 0.0589 - val_loss: 0.0446 - val_last_time_step_mse: 0.0352
Epoch 2/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0392 - last_time_step_mse: 0.0312 - val_loss: 0.0343 - val_last_time_step_mse: 0.0263
Epoch 3/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0311 - last_time_step_mse: 0.0224 - val_loss: 0.0280 - val_last_time_step_mse: 0.0183
Epoch 4/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0265 - last_time_step_mse: 0.0161 - val_loss: 0.0249 - val_last_time_step_mse: 0.0143
Epoch 5/20
219/219 [==============================] - 2s 9ms/step - loss: 0.0244 - last_time_step_mse: 0.0133 - val_loss: 0.0235 - val_last_time_step_mse: 0.0127
Epoch 6/20
219/219 [==============================] - 7s 30ms/step - loss: 0.0231 - last_time_step_mse: 0.0119 - val_loss: 0.0226 - val_last_time_step_mse: 0.0118
Epoch 7/20
219/219 [======

### 1维卷积神经网络，并对随机数据进行处理

In [9]:
# 卷积层
filters = 1 # 卷积核数量为 1
kernel_size = 5 # 卷积核大小为 5
convolution_1d_layer = Conv1D(filters, kernel_size, 
    strides=1, padding='valid', input_shape=(25, 1), 
    activation="relu")
# 池化层
max_pooling_layer = MaxPool1D(pool_size=5, strides=1, 
    padding="valid")

model1 = keras.models.Sequential([
    convolution_1d_layer,
    max_pooling_layer
])
# 创建 1 维向量，并扩展维度为 (1, 25, 1)
data_1d = np.random.normal(size=25)
data_1d = np.expand_dims(data_1d, 0)
data_1d = np.expand_dims(data_1d, 2)
print(data_1d.shape)
output = model1.predict(data_1d)
print(output)
print(output.shape)

(1, 25, 1)
[[[1.6615312 ]
  [1.6615312 ]
  [1.3948901 ]
  [1.3948901 ]
  [1.3948901 ]
  [1.3948901 ]
  [1.3948901 ]
  [0.79422855]
  [0.65266365]
  [0.65266365]
  [1.9572995 ]
  [1.9572995 ]
  [2.0132625 ]
  [2.0132625 ]
  [2.0132625 ]
  [2.0132625 ]
  [2.7978804 ]]]
(1, 17, 1)


In [10]:
model1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 21, 1)             6         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 17, 1)             0         
Total params: 6
Trainable params: 6
Non-trainable params: 0
_________________________________________________________________


In [ ]:
output.shape

(1, 25, 10)

### WaveNet

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.InputLayer(input_shape=[None, 1]))
for rate in (1, 2, 4, 8) * 2:
    model.add(keras.layers.Conv1D(filters=20, kernel_size=2, padding="causal",
                                  activation="relu", dilation_rate=rate))
model.add(keras.layers.Conv1D(filters=10, kernel_size=1))
model.compile(loss="mse", optimizer="adam", metrics=[last_time_step_mse])
history = model.fit(X_train, Y_train, epochs=20,
                    validation_data=(X_valid, Y_valid))

Epoch 1/20
219/219 [==============================] - 4s 13ms/step - loss: 0.0651 - last_time_step_mse: 0.0524 - val_loss: 0.0376 - val_last_time_step_mse: 0.0227
Epoch 2/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0321 - last_time_step_mse: 0.0187 - val_loss: 0.0295 - val_last_time_step_mse: 0.0160
Epoch 3/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0278 - last_time_step_mse: 0.0149 - val_loss: 0.0271 - val_last_time_step_mse: 0.0139
Epoch 4/20
219/219 [==============================] - 2s 11ms/step - loss: 0.0259 - last_time_step_mse: 0.0132 - val_loss: 0.0260 - val_last_time_step_mse: 0.0130
Epoch 5/20
219/219 [==============================] - 2s 11ms/step - loss: 0.0249 - last_time_step_mse: 0.0124 - val_loss: 0.0247 - val_last_time_step_mse: 0.0119
Epoch 6/20
219/219 [==============================] - 3s 12ms/step - loss: 0.0240 - last_time_step_mse: 0.0116 - val_loss: 0.0238 - val_last_time_step_mse: 0.0110
Epoch 7/20
219/219 [==